# Setup Visdom

Install it with:

`pip install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.


# LDA Training Visualization

To monitor the LDA training, a list of Metrics can be passed to the LDA function call for plotting their values live as the training progresses.  

Let's plot the training stats for an LDA model being trained on Lee corpus. We will use the four evaluation metrics available for topic models in gensim: Coherence, Perplexity, Topic diff and Convergence. (using separate hold_out and test corpus for evaluating the perplexity)

In [10]:
import os
import re
import logging
import gensim
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

def read_corpus(fname):
    texts = []
    with open(fname, encoding="ISO-8859-1") as f:
        for line in f:
            # lower case all words
            lowered = line.lower()
            # remove punctuation and split into seperate words
            words = re.compile('\w+').findall(lowered)
            texts.append(words)
    return texts

training_texts = read_corpus(lee_train_file)
test_texts = read_corpus(lee_test_file)

# Split test data into hold_out and test corpus
holdout_texts = test_texts[:25]
test_texts = test_texts[25:]

dictionary = Dictionary(training_texts)

training_corpus = [dictionary.doc2bow(text) for text in training_texts]
holdout_corpus = [dictionary.doc2bow(text) for text in holdout_texts]
test_corpus = [dictionary.doc2bow(text) for text in test_texts]




texts = [['bank','river','shore','water'],
        ['river','water','flow','fast','tree'],
        ['bank','water','fall','flow'],
        ['bank','bank','water','rain','river'],
        ['river','water','mud','tree'],
        ['money','transaction','bank','finance'],
        ['bank','borrow','money'], 
        ['bank','finance'],
        ['finance','money','sell','bank'],
        ['borrow','sell'],
        ['bank','loan','sell']]

training_texts = texts[:5]
test_texts = texts[5:]

dictionary = Dictionary(training_texts)

training_corpus = [dictionary.doc2bow(text) for text in training_texts]
test_corpus = [dictionary.doc2bow(text) for text in test_texts]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(7194 unique tokens: ['payment', 'protesters', 'responded', 'scrapper', 'daunting']...) from 300 documents (total 61260 corpus positions)
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(10 unique tokens: ['water', 'fall', 'river', 'rain', 'fast']...) from 5 documents (total 22 corpus positions)


In [11]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback for hold_out and test corpus
# pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity (hold_out)")
pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity (test)")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="visdom", viz_env="LdaModel", title="Coherence (u_mass)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", viz_env="LdaModel", title="Diff (kullback_leibler)")
convergence_jc = ConvergenceMetric(distance="jaccard", logger="visdom", viz_env="LdaModel", title="Convergence (jaccard)")

callbacks = [pl_test, ch_umass, diff_kl, convergence_jc]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=dictionary, passes=5, num_topics=5, callbacks=callbacks)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using symmetric eta at 0.1
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 5 topics, 5 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-4.740 per-word bound, 26.7 perplexity estimate based on a held-out corpus of 5 documents with 22 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #5/5
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 5 documents
DEBUG:gensim.models.ldamodel:5/5 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.314*"bank" + 0.172*"river" + 0.171*"water" + 0.171*"rain" + 0.029*"t

INFO:gensim.models.ldamodel:topic #1 (0.200): 0.200*"water" + 0.200*"flow" + 0.200*"fall" + 0.200*"bank" + 0.033*"river" + 0.033*"tree" + 0.033*"rain" + 0.033*"shore" + 0.033*"fast" + 0.033*"mud"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.200*"water" + 0.200*"river" + 0.200*"tree" + 0.200*"mud" + 0.033*"bank" + 0.033*"flow" + 0.033*"fall" + 0.033*"fast" + 0.033*"rain" + 0.033*"shore"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.200*"water" + 0.200*"river" + 0.200*"shore" + 0.200*"bank" + 0.033*"tree" + 0.033*"flow" + 0.033*"fall" + 0.033*"rain" + 0.033*"mud" + 0.033*"fast"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.171*"river" + 0.171*"water" + 0.171*"tree" + 0.171*"fast" + 0.171*"flow" + 0.029*"bank" + 0.029*"fall" + 0.029*"shore" + 0.029*"mud" + 0.029*"rain"
INFO:gensim.models.ldamodel:topic diff=0.002968, rho=0.447214
DEBUG:gensim.models.ldamodel:bound: at document #0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost
DEBUG:urllib3.connectionpool:h

When the model is set for training, you can open http://localhost:8097 to see the training progress.

In [3]:
# to get a metric value on a trained model
print(CoherenceMetric(corpus=training_corpus, coherence="u_mass").get_value(model=model))

-21.8411539992


The four types of graphs which are plotted for LDA:

**Coherence**

Coherence is a measure used to evaluate topic models. A good model will generate coherent topics, i.e., topics         with high topic coherence scores. Good topics are topics that can be described by a short label based on the topic     terms they spit out. 

<img src="Coherence.gif">

Now, this graph along with the others explained below, can be used to decide if it's time to stop the training. We     can see if the value stops changing after some epochs and that we are able to get the highest possible coherence       of our model.  


**Perplexity**

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. In LDA, topics are described by a probability distribution over vocabulary words. So, perplexity can be used to compare probabilistic models like LDA.

<img src="Perplexity.gif">

For a good model, perplexity should be as low as possible.


**Topic Difference**

Topic Diff calculates the distance between two LDA models. This distance is calculated based on the topics, by either using their probability distribution over vocabulary words (kullback_leibler, hellinger) or by simply using the common vocabulary words between the topics from both model.

<img src="Diff.gif">

In the heatmap, X-axis define the Epoch no. and Y-axis define the distance between the identical topic from consecutive epochs. For ex. a particular cell in the heatmap with values (x=3, y=5, z=0.4) represent the distance(=0.4) between the topic 5 from 3rd epoch and topic 5 from 2nd epoch. With increasing epochs, the distance between the identical topics should decrease.
  
  
**Convergence**

Convergence is the sum of the difference between all the identical topics from two consecutive epochs. It is basically the sum of column values in the heatmap above.

<img src="Convergence.gif">

The model is said to be converged when the convergence value stops descending with increasing epochs.

# Training Logs

We can also log the metric values to the shell apart from visualizing them in Visdom. The only change is to define `logger="shell"` instead of `"visdom"` in the input callbacks.

In [9]:
# define perplexity callback for hold_out and test corpus
# pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="shell")
pl_test = PerplexityMetric(corpus=test_corpus, logger="shell")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="shell")
diff_kl = DiffMetric(distance="kullback_leibler", logger="shell")
convergence_jc = ConvergenceMetric(distance="jaccard", logger="shell")

callbacks = [pl_test, ch_umass, diff_kl, convergence_jc]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=dictionary, passes=5, num_topics=5, callbacks=callbacks)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using symmetric eta at 0.1
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 5 topics, 5 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-4.742 per-word bound, 26.8 perplexity estimate based on a held-out corpus of 5 documents with 22 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #5/5
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 5 documents
DEBUG:gensim.models.ldamodel:4/5 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.171*"river" + 0.171*"flow" + 0.171*"water" + 0.171*"tree" + 0.171*"f

DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.172*"flow" + 0.172*"tree" + 0.171*"river" + 0.171*"water" + 0.171*"fast" + 0.029*"bank" + 0.029*"shore" + 0.029*"mud" + 0.029*"fall" + 0.029*"rain"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.100*"water" + 0.100*"river" + 0.100*"bank" + 0.100*"tree" + 0.100*"flow" + 0.100*"shore" + 0.100*"fall" + 0.100*"mud" + 0.100*"fast" + 0.100*"rain"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.315*"bank" + 0.171*"river" + 0.171*"water" + 0.171*"rain" + 0.029*"tree" + 0.029*"fast" + 0.029*"flow" + 0.029*"shore" + 0.029*"fall" + 0.029*"mud"
INFO:gensim.models.ldamodel:topic #3 (0.200): 0.229*"water" + 0.157*"river" + 0.157*"bank" + 0.086*"mud" + 0.086*"fall" + 0.086*"shore" + 0.086*"flow" + 0.086*"tree" + 0.014*"fast" + 0.014*"rain"
INFO:gensim.models.ldamodel:topic #4 (0.200): 0.100*"water" + 0.100*"bank" + 0.100*"tree" + 0.100*"river" + 0.100*"flow" + 0.100*"fall" + 0.100*"shore" + 0.100*"mud" + 0.1